# Objective

This notebook shows the connection to AWS and a Hello World with some of the services used

1) S3
2) Lambda (try the hello world from midterm)
3) Cloud Watch

To connect a key was made using IAM, and put into a local `.env` file, the credentials are temporary and will expire in 30 days. Care should be used when making these and different keys used for dev and prod.


* outside POC environment consider different security practices such as SSO 


In [2]:
# !pip install pydot
# !pip install python-dotenv

In [3]:
import sys

import boto3
from botocore.config import Config
from dotenv import dotenv_values


config = dotenv_values(".env") 

print(sys.version)

3.11.8 (v3.11.8:db85d51d3e, Feb  6 2024, 18:02:37) [Clang 13.0.0 (clang-1300.0.29.30)]


In [4]:
# connect to S3
s3_resource = boto3.client(
    's3',
    aws_access_key_id=config["aws_access_key_id"],
    aws_secret_access_key=config["aws_secret_access_key"]
)


d = s3_resource.list_buckets()

# show current buckets
b = [n["Name"] for n in d["Buckets"]]

b

['aws-athena-query-results-211125778552-us-east-1',
 'cosmicai-data',
 'cosmicai2',
 'fmi-lambda-demo',
 'group2-s3-bucket',
 'group4-s3-bucket',
 'sagemaker-studio-211125778552-3zpozdpwzcx',
 'sagemaker-studio-211125778552-rrp76qgcj1n',
 'sagemaker-us-east-1-211125778552',
 'team-one-cosmic-data',
 'team-one-s3-cosmic',
 'team2cosmicai',
 'team3cosmicai',
 'team4-cosmicai']

In [5]:
# validate folders needed in connection
assert 'fmi-lambda-demo' in b, "missing our team4 cosmicai S3 connection"
assert 'team4-cosmicai' in b, "missing our team4 cosmicai S3 connection"


In [6]:
set_region = Config(
    region_name = config["region"]
)

# Create CloudWatch client
cloudwatch = boto3.client('cloudwatch', 
                          aws_access_key_id=config["aws_access_key_id"],
                          aws_secret_access_key=config["aws_secret_access_key"],
                          config = set_region)


In [19]:
# List metrics through the pagination interface

paginator = cloudwatch.get_paginator('list_metrics')
for response in paginator.paginate(Dimensions=[{'Name': 'LogGroupName'}],
                                   MetricName='IncomingLogEvents',
                                   Namespace='AWS/Logs'):
    print(response['Metrics'])

[{'Namespace': 'AWS/Logs', 'MetricName': 'IncomingLogEvents', 'Dimensions': [{'Name': 'LogGroupName', 'Value': '/aws/sagemaker/ProcessingJobs'}]}, {'Namespace': 'AWS/Logs', 'MetricName': 'IncomingLogEvents', 'Dimensions': [{'Name': 'LogGroupName', 'Value': '/aws/vendedlogs/states/DS5110StateMachine2-Logs'}]}, {'Namespace': 'AWS/Logs', 'MetricName': 'IncomingLogEvents', 'Dimensions': [{'Name': 'LogGroupName', 'Value': '/aws/vendedlogs/states/DS5110StateMachine3-Logs'}]}, {'Namespace': 'AWS/Logs', 'MetricName': 'IncomingLogEvents', 'Dimensions': [{'Name': 'LogGroupName', 'Value': '/aws/lambda/cosmic-init'}]}, {'Namespace': 'AWS/Logs', 'MetricName': 'IncomingLogEvents', 'Dimensions': [{'Name': 'LogGroupName', 'Value': '/aws/sagemaker/studio'}]}, {'Namespace': 'AWS/Logs', 'MetricName': 'IncomingLogEvents', 'Dimensions': [{'Name': 'LogGroupName', 'Value': '/aws/lambda/resultSummary'}]}, {'Namespace': 'AWS/Logs', 'MetricName': 'IncomingLogEvents', 'Dimensions': [{'Name': 'LogGroupName', 'Val

In [35]:
# show the log groups
import pandas as pd

pd.json_normalize(response, record_path=["Metrics", "Dimensions"])

,Name,Value
0,LogGroupName,/aws/sagemaker/ProcessingJobs
1,LogGroupName,/aws/vendedlogs/states/DS5110StateMachine2-Logs
2,LogGroupName,/aws/vendedlogs/states/DS5110StateMachine3-Logs
3,LogGroupName,/aws/lambda/cosmic-init
4,LogGroupName,/aws/sagemaker/studio
5,LogGroupName,/aws/lambda/resultSummary
6,LogGroupName,/aws/lambda/data-parallel-init
7,LogGroupName,/aws/vendedlogs/states/Group2_cosmicai-Logs
8,LogGroupName,/ecs/rendezvous-tcpunch-fargate-task
9,LogGroupName,/aws/vendedlogs/states/DataParallel-CosmicAI-Logs


In [ ]:
# TODO: get out metrics from runs\
    # pipe into data so team can use

# TODO: try a lambda function hello world